# Imports

In [ ]:
import pandas as pd 
import numpy as np

### Base com atributos numéricos

In [ ]:
# Base tirada de https://archive.ics.uci.edu/ml/datasets/Ecoli

ecoli = pd.read_csv('databases/ecoli.csv',
                      header=None, 
                      names=[
                        "Sequence Name",
                        "mcg",
                        "gvh",
                        "lip",
                        "chg",
                        "aac",
                        "alm1",
                        "alm2",
                        "Class"
                      ])

# Substituindo a string de pendência para valores compreenciveis
# ecoli = ecoli.replace({

# })

ecoli

In [ ]:
# ecoli.describe()

In [ ]:
# ecoli.dtypes

## Pré-Processamento Inicial

##### Deletando a coluna 'Sequence Name'

In [ ]:
# A coluna em questão é uma coluna de ID de busca, logo não é necessária para o que será feito no trabalho
if ecoli.get("Sequence Name") is not None:
    del ecoli["Sequence Name"]

ecoli.dtypes

In [ ]:
from sklearn.preprocessing import StandardScaler

atributos = [           
    "mcg",
    "gvh",
    "lip",
    "chg",
    "aac",
    "alm1",
    "alm2",
    ]

ecoli[atributos] = StandardScaler().fit_transform(ecoli[atributos])
#ecoli = pd.DataFrame(balance)

limiar = 7

ecoli

## Aprendizagem supervisionada
##### Selecionando o X e y

In [ ]:
# Selecionando o X e o y


# X = ecoli[atributos]
X = ecoli.iloc[:, :limiar] # Atributos do Data Frame do Pandas
y = ecoli.iloc[:, limiar:] # Classes do Data Frame do Pandas

y = np.squeeze(y) # Converter em um unico Array quando usar o dataFrame do Pandas

y_unique = np.unique(y)

from sklearn.preprocessing import LabelEncoder # Criando um mapa de cores para mapear o gráfico

lb = LabelEncoder()
lb.fit(y)
mapa_cores = lb.transform(y)
mapa_cores
# y_unique # Valores unicos de classes

##### PCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print('Projecting %d-dimensional data to 2D' % X.shape[1])

def plot_g(pyplot, title, colors, n_classes):
    pyplot.scatter(X_reduced[:, 0], X_reduced[:, 1], c=colors, 
             edgecolor='none', alpha=0.7, s=40,
             cmap=plt.cm.get_cmap('nipy_spectral', n_classes)
           )
    pyplot.colorbar()
    pyplot.title(title)

plt.figure(figsize=(10, 8)) # Tamanho da Figura
plot_g(plt, 'Projeção 2D do Dados em PCA', mapa_cores, len(y_unique))
plt.show()


##### Plotando os Gráficos 

In [ ]:

# plt.figure(figsize=(10*2, 7))

# plt.subplot(1, 2, 1) # row 1, col 2 index 1
# plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels, 
#             edgecolor='none', alpha=0.7, s=40,
#             cmap=plt.cm.get_cmap('nipy_spectral', len(y_unique)))
# plt.colorbar()
# plt.title('Projeção KMeans com X Original')

# plt.subplot(1, 2, 2) # row 1, col 2 index 1
# plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels_PCA, 
#             edgecolor='none', alpha=0.7, s=40,
#             cmap=plt.cm.get_cmap('nipy_spectral', len(y_unique)))
# plt.colorbar()
# plt.title('Projeção KMeans com X PCA')

# plt.show()

# Projeto
##### A seguir é feita uma seleção aleátoria dos dados para cada algorítmo com KFold e em seguida o treinamento e avaliação de cada método. A ordem será a estimação dos parâmetros de (kNN, SVM, MLP), e para cada estimação é selecionada avaliação do melhor parâmetro para cada algorítmo. Para o KNN o melhor numero de visinhos dentre [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], para o SVM o melhor kernel dentre ["linear", "poly", "rbf", "sigmoid"], e por fim para a rede neural a melhor função de ativação ["identity", "logistic", "tanh", "relu"] respectivamente.
### 1. kNN

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


from sklearn.neighbors import KNeighborsClassifier

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

valores_k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
resultados_knn = []

for k in valores_k:
    resultados_k = []
    knn = KNeighborsClassifier(n_neighbors=k)
    kf = KFold(n_splits=10, shuffle=True)
    for index_train, index_valid in kf.split(X_treino):
        X_train, X_validacao = X_treino.iloc[index_train], X_treino.iloc[index_valid]
        y_train, y_validacao = y_treino.iloc[index_train], y_treino.iloc[index_valid]
        knn.fit(X_train, y_train)
        resultados_k.append(knn.score(X_validacao, y_validacao))
    resultados_knn.append(sum(resultados_k)/len(resultados_k))
# resultados_knn

big_value = np.max(resultados_knn)

mapped_values = dict(zip(valores_k, resultados_knn))

better_knn = 0

for m in mapped_values:
    if mapped_values[m] == big_value:
        better_knn = m
        break

print(dict(zip(["O melhor Knn atual", "Maior valor"], [better_knn, big_value])))

mapped_values

##### O melhor k escolhido é agora usado  na classificação real do Knn

In [ ]:
# Desempenho real do Knn

knn = KNeighborsClassifier(n_neighbors=better_knn)
knn.fit(X_treino, y_treino)
knn_score = knn.score(X_teste, y_teste)
knn_score

##### Avaliação por Precisão, Recall e F1

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = knn.predict(X_teste)

score_p = precision_score(y_teste, y_pred, average='macro')
score_r = recall_score(y_teste, y_pred, average='macro')
score_f1 = f1_score(y_teste, y_pred, average='macro')

pd.DataFrame([score_p, score_r, score_f1], index = ["Precision", "Recall", "F1"], columns=["Scores"])

### 2. SVM

In [ ]:
from sklearn.svm import SVC
valores_kernel = ["linear", "poly", "rbf", "sigmoid"]
resultados_svm = []

for kernel in valores_kernel:
    resultados_kernel = []
    svc = SVC(kernel=kernel)
    kf = KFold(n_splits=10, shuffle=True)
    for index_train, index_valid in kf.split(X_treino):
        X_train, X_validacao = X_treino.iloc[index_train], X_treino.iloc[index_valid]
        y_train, y_validacao = y_treino.iloc[index_train], y_treino.iloc[index_valid]
        svc.fit(X_train, y_train)
        resultados_kernel.append(svc.score(X_validacao, y_validacao))
    resultados_svm.append(sum(resultados_kernel)/len(resultados_kernel))


big_value = np.max(resultados_svm)

mapped_values = dict(zip(valores_kernel, resultados_svm))

better_k = ""

for m in mapped_values:
    if mapped_values[m] == big_value:
        better_k = m
        break

print(dict(zip(["O melhor Kernel atual", "Maior valor"], [better_k, big_value])))

mapped_values

##### O melhor Kernel escolhido é usado na classificação real do SVM

In [ ]:
svc = SVC(kernel=better_k)
svc.fit(X_treino, y_treino)
svc_score = svc.score(X_teste, y_teste)
svc_score

##### Avaliação por Precisão, Recall e F1

In [ ]:
y_pred = svc.predict(X_teste)

score_p = precision_score(y_teste, y_pred, average='macro')
score_r = recall_score(y_teste, y_pred, average='macro')
score_f1 = f1_score(y_teste, y_pred, average='macro')

pd.DataFrame([score_p, score_r, score_f1], index = ["Precision", "Recall", "F1"], columns=["Scores"])

### 3. MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
valores_ativacao = ["identity", "logistic", "tanh", "relu"]
resultados_mlp = []

for ativacao in valores_ativacao:
    resultados_ativacao = []
    mlp = MLPClassifier(activation=ativacao)
    kf = KFold(n_splits=10, shuffle=True)
    for index_train, index_valid in kf.split(X_treino):
        X_train, X_validacao = X_treino.iloc[index_train], X_treino.iloc[index_valid]
        y_train, y_validacao = y_treino.iloc[index_train], y_treino.iloc[index_valid]
        mlp.fit(X_train, y_train)
        resultados_ativacao.append(svc.score(X_validacao, y_validacao))
    resultados_mlp.append(sum(resultados_ativacao)/len(resultados_ativacao))



big_value = np.max(resultados_mlp)

mapped_values = dict(zip(valores_ativacao, resultados_mlp))

better_a = ""

for m in mapped_values:
    if mapped_values[m] == big_value:
        better_a = m
        break

print(dict(zip(["O melhor parametro de ativação atual", "Maior valor"], [better_a, big_value])))
mapped_values

##### O melhor Valor de ativação escolhido é usado na classificação real do MLP

In [ ]:
mlp = MLPClassifier(activation=better_a)
mlp.fit(X_treino, y_treino)
mlp_score = mlp.score(X_teste, y_teste)
mlp_score

##### Avaliação por Precisão, Recall e F1

In [ ]:
y_pred = mlp.predict(X_teste)

score_p = precision_score(y_teste, y_pred, average='macro')
score_r = recall_score(y_teste, y_pred, average='macro')
score_f1 = f1_score(y_teste, y_pred, average='macro')

pd.DataFrame([score_p, score_r, score_f1], index = ["Precision", "Recall", "F1"], columns=["Scores"])